# 0.1.0 Introduction

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import plotly.express as px

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings("ignore")

In [3]:
import sys
sys.path.append("../") 

import utils.paths as path
from utils.paths2 import direcciones

## 0.1.2 Importing headcount and turnover data

In [4]:
org = pd.read_csv(path.data_raw_dir("org.csv"), sep=',')
print(org.info())
org.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2291 entries, 0 to 2290
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   emp_id             2291 non-null   object 
 1   status             2291 non-null   object 
 2   turnover           2291 non-null   int64  
 3   location           2291 non-null   object 
 4   level              2291 non-null   object 
 5   date_of_joining    2291 non-null   object 
 6   date_of_birth      2291 non-null   object 
 7   last_working_date  410 non-null    object 
 8   gender             2291 non-null   object 
 9   department         2291 non-null   object 
 10  mgr_id             2291 non-null   object 
 11  cutoff_date        2291 non-null   object 
 12  generation         2291 non-null   object 
 13  emp_age            2291 non-null   float64
dtypes: float64(1), int64(1), object(12)
memory usage: 250.7+ KB
None


,emp_id,status,turnover,location,level,date_of_joining,date_of_birth,last_working_date,gender,department,mgr_id,cutoff_date,generation,emp_age
0,E11061,Inactive,1,New York,Analyst,22/03/2012,22/03/1992,11/09/2014,Male,Customer Operations,E1712,31/12/2014,Millennials,22.5
1,E1031,Inactive,1,New York,Analyst,09/03/2012,10/01/1992,05/06/2014,Female,Customer Operations,E10524,31/12/2014,Millennials,22.4
2,E6213,Inactive,1,New York,Analyst,06/01/2012,06/02/1992,30/04/2014,Female,Customer Operations,E4443,31/12/2014,Millennials,22.2
3,E5900,Inactive,1,New York,Analyst,22/03/2012,19/12/1991,09/04/2014,Female,Customer Operations,E3638,31/12/2014,Millennials,22.3
4,E3044,Inactive,1,Florida,Analyst,29/03/2012,10/12/1991,23/01/2014,Female,Customer Operations,E3312,31/12/2014,Millennials,22.1


In [5]:
org.describe()

,turnover,emp_age
count,2291.000000,2291.000000
mean,0.178961,29.904452
std,0.383403,4.555086
min,0.000000,22.100000
25%,0.000000,26.300000
50%,0.000000,29.000000
75%,0.000000,32.900000
max,1.000000,58.000000


## 0.1.3 What proportion of employees have left?

In [6]:
org['status'].value_counts()

Active      1881
Inactive     410
Name: status, dtype: int64

In [7]:
org['status'] = np.where(org['status'] == 'Active', 0, 1)
np.mean(org['status'])

0.1789611523352248

So approximately 18% of the employees left the organization.

## 0.1.4 Which levels have high turnover rate?

In [8]:
df_level = org.groupby('level')['status'].mean().reset_index()
df_level

,level,status
0,Analyst,0.215087
1,Assistant Manager,0.036458
2,Director,0.000000
3,Manager,0.043478
4,Senior Manager,0.000000
5,Specialist,0.148571
6,Vice President,0.000000


In [9]:
fig = px.bar(df_level, x='level', y='status')
fig.show()

The graph represents a high turnover rate at the Analyst and Specialist levels as compared to other levels.

## 0.1.5 Is turnover rate different across locations?

In [10]:
df_location = org.groupby('location')['status'].mean().reset_index()
df_location

,location,status
0,Chicago,0.325641
1,Florida,0.105513
2,New York,0.202591


In [11]:
fig = px.bar(df_location, x='location', y='status')
fig.show()

Turnover rate in Chicago is high when compared to other locations.

## 0.1.7 Filtering the dataset

In [12]:
org['level'].value_counts()

Analyst              1604
Specialist            350
Assistant Manager     192
Manager               138
Senior Manager          5
Director                1
Vice President          1
Name: level, dtype: int64

In [13]:
org['level'].unique()

array(['Analyst', 'Assistant Manager', 'Specialist', 'Manager',
       'Senior Manager', 'Director', 'Vice President'], dtype=object)

In [14]:
org2 = org.copy()
org2 = org2[(org2['level'] == 'Analyst') | (org2['level'] == 'Specialist')]  
org2['level'].value_counts()

Analyst       1604
Specialist     350
Name: level, dtype: int64

## 0.1.8 Combining HR datasets (I)

In [15]:
rating = pd.read_csv(path.data_raw_dir("rating.csv"), sep=',')
print(rating.info())
rating.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1954 entries, 0 to 1953
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   emp_id  1954 non-null   object
 1   rating  1954 non-null   object
dtypes: object(2)
memory usage: 30.7+ KB
None


,emp_id,rating
0,E8,Acceptable
1,E9,Acceptable
2,E12,Acceptable
3,E15,Acceptable
4,E34,Acceptable


In [16]:
org3 = pd.merge(org2, rating, how='left', on=['emp_id']) 

In [17]:
df_rating = org3.groupby('rating')['turnover'].mean().reset_index()
df_rating

,rating,turnover
0,Above Average,0.131295
1,Acceptable,0.220974
2,Below Average,0.384615
3,Excellent,0.030534
4,Unacceptable,0.633333


## 0.1.9 Combining HR datasets (II)

In [18]:
survey = pd.read_csv(path.data_raw_dir("survey.csv"), sep=',')
print(survey.info())
survey.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   mgr_id               350 non-null    object 
 1   mgr_effectiveness    350 non-null    float64
 2   career_satisfaction  350 non-null    float64
 3   perf_satisfaction    350 non-null    float64
 4   work_satisfaction    350 non-null    float64
dtypes: float64(4), object(1)
memory usage: 13.8+ KB
None


,mgr_id,mgr_effectiveness,career_satisfaction,perf_satisfaction,work_satisfaction
0,E1003,0.76,0.76,0.71,0.82
1,E10072,0.65,0.67,0.56,0.84
2,E10081,0.80,0.82,0.73,0.84
3,E10234,0.65,0.63,0.75,0.70
4,E1026,0.70,1.00,1.00,0.92


In [19]:
org_final = pd.merge(org3, survey, how='left', on=['mgr_id']) 

In [20]:
fig = px.box(org_final, x='status', y="mgr_effectiveness")
fig.update_xaxes(
    tickvals=[0,1],
    ticktext=['Active', 'Inactive'])
fig.show()

## 0.1.10 Master data overview

In [21]:
org_final = pd.read_csv(path.data_raw_dir("org_final.csv"), sep=',')
print(org_final.info())
org_final.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1954 entries, 0 to 1953
Data columns (total 34 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   emp_id                  1954 non-null   object 
 1   status                  1954 non-null   object 
 2   location                1954 non-null   object 
 3   level                   1954 non-null   object 
 4   gender                  1954 non-null   object 
 5   emp_age                 1954 non-null   float64
 6   rating                  1954 non-null   object 
 7   mgr_rating              1954 non-null   object 
 8   mgr_reportees           1954 non-null   int64  
 9   mgr_age                 1954 non-null   float64
 10  mgr_tenure              1954 non-null   float64
 11  compensation            1954 non-null   int64  
 12  percent_hike            1954 non-null   int64  
 13  hiring_score            1954 non-null   int64  
 14  hiring_source           1954 non-null   

,emp_id,status,location,level,gender,emp_age,rating,mgr_rating,mgr_reportees,mgr_age,mgr_tenure,compensation,percent_hike,hiring_score,hiring_source,no_companies_worked,distance_from_home,total_dependents,marital_status,education,promotion_last_2_years,no_leaves_taken,total_experience,monthly_overtime_hrs,date_of_joining,last_working_date,department,mgr_id,cutoff_date,turnover,mgr_effectiveness,career_satisfaction,perf_satisfaction,work_satisfaction
0,E10012,Active,New York,Analyst,Female,25.09,Above Average,Acceptable,9,44.07,3.17,64320,10,70,Consultant,1,14,2,Single,Bachelors,No,2,6.86,1,6/03/2011,NaN,Customer Operations,E9335,31/12/2014,0,0.730,0.73,0.73,0.75
1,E10025,Active,Chicago,Analyst,Female,25.98,Acceptable,Excellent,4,35.99,7.92,48204,8,70,Job Fairs,9,21,2,Single,Bachelors,No,10,4.88,5,23/09/2009,NaN,Customer Operations,E6655,31/12/2014,0,0.581,0.72,0.84,0.85
2,E10027,Active,Orlando,Specialist,Female,33.40,Acceptable,Above Average,6,35.78,4.38,85812,11,77,Consultant,3,15,5,Single,Bachelors,Yes,18,8.55,3,2/11/2005,NaN,Customer Operations,E13942,31/12/2014,0,0.770,0.85,0.80,0.87
3,E10048,Active,Chicago,Specialist,Male,24.55,Acceptable,Acceptable,10,26.70,2.87,49536,8,71,Job Boards,5,9,3,Single,Bachelors,Yes,19,4.76,8,20/05/2011,NaN,Customer Operations,E7063,31/12/2014,0,0.240,0.42,0.33,0.85
4,E10060,Active,Orlando,Analyst,Male,31.23,Acceptable,Acceptable,11,34.28,12.95,75576,12,70,Job Fairs,1,25,4,Single,Bachelors,No,25,8.06,1,21/12/2011,NaN,Customer Operations,E5663,31/12/2014,0,0.710,0.78,0.67,0.80


In [22]:
fig = px.box(org_final, x='status', y="distance_from_home")
fig.update_xaxes(
    tickvals=[0,1],
    ticktext=['Active', 'Inactive'])
fig.show()

In [23]:
print('ok_')

ok_
